In [1]:
#basic package
import json
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import shutil
import time
import datetime as dt
import itertools
import time
import re
from collections import Counter
from collections import defaultdict
import operator
import math
import functools
import collections
import random

#generate random id
import uuid

#ICC in python
import pingouin as pg

#modelling
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix, cohen_kappa_score
from xgboost import plot_importance
#catboost, for a better support of categorical data
from catboost import CatBoostClassifier, Pool
#to use all except one core
import multiprocessing

#plot
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
from UTILS import is_WG_open, ConfMat, list_of_zones, res_intodico
import config_origins as config

C:\Users\camil\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


# Download Input Data

In [3]:
#define parameters
path_extracted_data = config.path_extracted_data
path_initial_data = config.path_initial_data
id_run = config.id_run
dico_matching = config.dico_matching
dico_garden_opening_hour = config.dico_garden_opening_hour
path_extracted_data_cleaning = os.path.join(path_extracted_data,'Cleaning')
path__ = os.path.join(path_extracted_data_cleaning,'1secTS')
path_extracted_data_cleaning_model = os.path.join(path_extracted_data_cleaning,'model') 
if not os.path.exists(path_extracted_data_cleaning_model):
    os.makedirs(path_extracted_data_cleaning_model)    
#create a director if not existing
if not os.path.exists(path__):
    os.makedirs(path__)
    
#feature for model
li_cont_select = config.li_cont_select
li_bin = config.li_bin
li_cat_select = config.li_cat_select

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
df = pd.read_csv(os.path.join(path_extracted_data_cleaning, id_run+'InputCleaning.csv'), parse_dates=['Timestamp', 'date'], 
                 dayfirst=True, sep=';')
df['VideoAnalyse'] = df['VideoAnalyse'].map(lambda x: int(x)) #not before as tehre was string as well
df = df[df['VideoAnalyse'].isin([0,1])]
df = shuffle(df)
print(df.shape)
df.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(5137, 35)


,Timestamp,TagID,HenID,Trackingsystem_Zone,PenID,VideoAnalyse,system,date,type,signalstrength,...,previous_duration,algo_type,BatchID,previousZone==NextZone,next_duration,next2zone==Zone,previous2zone==Zone,zone3_match_exist,is_first_observation,Observed_Zone
232,2020-10-15 12:54:01,tag_143,hen_108,3_Zone,pen11,1,10 - 12,2020-10-15,students,11.0,...,20.0,verification,ID15,False,555.0,0.0,1.0,0.0,0.0,3_Zone
156,2020-10-25 12:55:46,tag_93,hen_105,3_Zone,pen9,1,8 - 9,2020-10-25,students,9.0,...,15.0,cleaning,NaN,True,22.0,1.0,1.0,1.0,0.0,3_Zone
4347,2020-10-24 15:52:32,tag_54,hen_21,3_Zone,pen5,1,3 - 5,2020-10-24,students,19.0,...,201.0,verification,ID38,True,24.0,0.0,1.0,1.0,0.0,3_Zone


# Data preprocessing

In [5]:
#scaling numerical data: necessary due to duration high values
display(df[li_cont_select].head(3))
sc = StandardScaler()
li_scale = [x for x in li_cont_select if x not in li_bin]
df[li_scale] = sc.fit_transform(df[li_scale])
display(df[li_cont_select].head(3))

#display categoricalfeatures
df[li_cat_select].head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,signalstrength,signalstzone2,duration,next_duration,previous_duration,next2zone==Zone,previous2zone==Zone,zone3_match_exist
232,11.0,9.0,59.0,555.0,20.0,0.0,1.0,0.0
156,9.0,6.0,68.0,22.0,15.0,1.0,1.0,1.0
4347,19.0,7.0,22.0,24.0,201.0,0.0,1.0,1.0


,signalstrength,signalstzone2,duration,next_duration,previous_duration,next2zone==Zone,previous2zone==Zone,zone3_match_exist
232,-0.158120,0.303697,-0.048633,0.010807,-0.037109,0.0,1.0,0.0
156,-0.654309,-0.482508,-0.047487,-0.056517,-0.037441,1.0,1.0,1.0
4347,1.826637,-0.220439,-0.053348,-0.056265,-0.025078,0.0,1.0,1.0


,Trackingsystem_Zone,zone2_match,previous1_zone,next1_zone,system
232,3_Zone,miniZone3,3_Zone,2_Zone,10 - 12
156,3_Zone,miniZone3,3_Zone,3_Zone,8 - 9
4347,3_Zone,3_Zone,3_Zone,3_Zone,3 - 5


# Split into training and validation (for parameter tuning) & dummies

In [6]:
df_cleaning = df[df['algo_type']=='cleaning'].copy()
df_cleaning['VideoAnalyse'].value_counts()

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1    3738
0     536
Name: VideoAnalyse, dtype: int64

In [ ]:
####### train, val, test (no dummie )#######
#we need to keep into dataframe format in order to have the columns names to specify in the hyperparamters the names of the
#categorical variables
#split into train, test and validation
#for algo from sklearn, we should add val+train together, while for the Catboost we will not add them up
#train
df_X_train_cb = df_cleaning.filter(li_cont_select+li_cat_select, axis=1)
df_y_train_cb = df_cleaning[['VideoAnalyse']]

#split training into validation: for algo from sklearn, we should add val+train together, while for the Catboost we will not 
#add them up
val_size = 0.2; rs = 0
df_X_train_cb, df_X_val_cb, df_y_train_cb, df_y_val_cb = train_test_split(df_X_train_cb, df_y_train_cb, test_size=val_size, 
                                                                          random_state=rs)
print(df_X_train_cb.shape, df_y_train_cb.shape, df_X_val_cb.shape, df_y_val_cb.shape)

# Catboost

In [18]:
li_dico = []

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
#Parameters fine tunning
#Info: https://catboost.ai/docs/concepts/python-reference_parameters-list.html
x1 = df_cleaning[df_cleaning['VideoAnalyse']==1].shape[0]
x0 = df_cleaning[df_cleaning['VideoAnalyse']==0].shape[0]
print(x1/x0)
P = {'n_estimators': range(200,1000,100), #the more trees you add, the higher the risk of overfitting!! default:1000!
     'l2_leaf_reg': range(2,5),#default is 3
     'loss_function': ['Logloss'], #for binary classification
     'max_depth': range(4,7,1), #if to big: risk of overfitting default=6
     'min_data_in_leaf': range(1,4), #The minimum number of training samples in a leaf (equivalent to min_samples_leaf). default 1
     'class_weights': [{0:x1/x0, 1:1}], #multiply the weight of object of class 0 by 1 and the object of class 1 by 0.2
     'od_pval': [10**-6],#[x for x in np.linspace(start = 10**-6, stop = 10**-2, num = 5)], #The larger the value, the earlier overfitting is detected., defualt=0, but it is recommanded to be tested for these range
     'od_wait':[10],#, #instead of the default 20 :Ignore the overfitting detector when the threshold is reached and continue learning for the specified number of iterations after the iteration with the optimal metric value.
     'use_best_model':[True],
     'task_type':['CPU'],#is actually faster
     'cat_features':[np.array(li_cat_select)]
    }
n_jobs = multiprocessing.cpu_count()-1 #all except one core
mod = GridSearchCV(estimator = CatBoostClassifier(), param_grid = P, cv = 3, verbose=True)
mod.fit(df_X_train_cb, df_y_train_cb, eval_set=(df_X_val_cb,df_y_val_cb), verbose=False)
mod_final = mod.best_estimator_
mod_final.fit(df_X_train_cb, df_y_train_cb, eval_set=(df_X_val_cb,df_y_val_cb), verbose=False)
display(mod_final.get_params())
print(mod_final.get_best_score())
#145.1min for 648 fits

6.973880597014926
Fitting 3 folds for each of 216 candidates, totalling 648 fits


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 648 out of 648 | elapsed: 96.0min finished


{'l2_leaf_reg': 3,
 'loss_function': 'Logloss',
 'od_pval': 1e-06,
 'od_wait': 10,
 'use_best_model': True,
 'class_weights': {0: 6.973880597014926, 1: 1},
 'task_type': 'CPU',
 'max_depth': 5,
 'n_estimators': 900,
 'cat_features': array(['Trackingsystem_Zone', 'zone2_match', 'previous1_zone',
        'next1_zone', 'system'], dtype='<U19'),
 'min_data_in_leaf': 1}

{'learn': {'Logloss': 0.0077482444410255065}, 'validation': {'Logloss': 0.02811191540683454}}


In [22]:
#save
mod_final.save_model(os.path.join(path_extracted_data_cleaning_model,'selected_Catboost'), format="cbm")

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
round(mod_final.score(df_X_train_cb, df_y_train_cb),3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.998

In [12]:
round(mod_final.score(df_X_val_cb, df_y_val_cb),3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.996

In [19]:
#save
li_dico.append({'algo':'best-Catboost', 'model_param_used':mod_final.get_params(), 
                'accuracy_training':round(mod_final.score(df_X_train_cb, df_y_train_cb),3),
                'accuracy_validation':round(mod_final.score(df_X_val_cb, df_y_val_cb),3)})
                #'accuracy_testing':round(mod_final.score(df_X_test_cb, df_y_test_cb),3)})
pd.DataFrame(li_dico)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,algo,model_param_used,accuracy_training,accuracy_validation
0,best-Catboost,"{'l2_leaf_reg': 3, 'loss_function': 'Logloss',...",0.998,0.996


# save final results

In [20]:
df_res = pd.DataFrame(li_dico)
print(df_res.shape)
df_res.to_csv(os.path.join(path_extracted_data_cleaning, 'CB_finetuning.csv'), sep=';', index=False)      

(1, 4)


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
path_extracted_data_cleaning

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'G:\\VPHI\\Welfare\\2- Research Projects\\OFHE2.OriginsE2\\DataOutput\\TrackingSystem\\chapter0_final_\\Cleaning'